# 反射
正式讲反射之前，先来说两个小方法，与反射没有关系，但是有点像

In [2]:
class Foo:
    pass
foo = Foo()
# 这是我们今天讲的，以后使用这个方式判断一个对象是不是类的对象
print(isinstance(foo,Foo))
# 这是我们以前判断的方式，type另有用途，以后不使用这中方式判断
print(type(foo) is Foo)
# 同理,一切皆对象
print(isinstance('abc',str))
# Python3 中的一个任何类都是object子类
print(issubclass(Foo,object))

True
True
True
True


## 一 反射内置函数 
下述四个函数是专门用来操作类与对象属性的，如何操作？通过字符串来操作类与对象的属性，这种操作称为反射

hasattr，getattr，setsttr，delattr，根据名字猜意思，attr是属性

之所以称之为反射，是因为把字符串映射成了一个具体的属性，因为我们平时是根据类或者对象来找属性的，而现在反过来找，所以叫反射。为什么我们要使用反射这种方式来对象的属性呢？从代码上看并没有简洁多少，主要是因为，反射是从字符串触发的，我们在获取用户输入的时候也是拿到的字符串，就在这里用。

In [4]:
class People:
    country="China"
    def __init__(self,name):
        self.name=name
    def tell(self):
        print('%s is aaa' %self.name)
obj=People('albert')
# 1、hasattr
print(hasattr(People,'country'))
# 本质原理就是下面这行
print('country' in People.__dict__)
# 同理，一切皆对象
print(hasattr(obj,'name'))
print(hasattr(obj,'country'))
print(hasattr(obj,'tell'))
print('###')
# 2、getattr
x=getattr(People,'country1',None)
# x=getattr(People,'country',None)
# x=getattr(People,'country1') # 如果没有这个属性，不传None会报错
print(x)
f=getattr(obj,'tell',None)  # obj.tell
print(f == obj.tell)
f()
obj.tell()
print('###')
# 3、setattr
# People.x=111
setattr(People,'x',111)
print(People.x)
# obj.age=18
setattr(obj,"age",18)
print(obj.__dict__)
print('###')
# 4、delattr
# del People.country
delattr(People,"country")
print(People.__dict__)
# del obj.name
delattr(obj,"name")
print(obj.__dict__)

True
True
True
True
True
###
None
True
albert is aaa
albert is aaa
###
111
{'name': 'albert', 'age': 18}
###
{'__module__': '__main__', '__init__': <function People.__init__ at 0x000002A11EDC1F28>, 'tell': <function People.tell at 0x000002A11EDC1B70>, '__dict__': <attribute '__dict__' of 'People' objects>, '__weakref__': <attribute '__weakref__' of 'People' objects>, '__doc__': None, 'x': 111}
{'age': 18}


## 二 面向对象内置方法
1. str方法
通过前面的学习，我想你已经很清楚一切皆对象的概念了，既然这样，现在有一个问题

In [5]:
class Foo:
    pass
obj = Foo()
l = [1, 2, 3, ]
print(obj)
print(l)

[1, 2, 3]


obj和l都是对象，为什么 l 能打印出来他的值，而obj不是打印打他的值而是打印一个内存地址出来
这说明你在在印 l 这个对象的时候一定是触发了某个方法，指定就让他打印对象的值，我们的类能不能也触发这个方法呢？
这个方法就是__str__方法，在对象打印时自动触发

In [6]:
class People:
    def __init__(self, name, age, sex):  # init对象实例化自动触发
        self.name = name
        self.age = age
        self.sex = sex
    def __str__(self):  # str对象打印时自动触发
        # print('========>')
        return '<名字：%s 年龄：%s 性别：%s>' % (self.name, self.age, self.sex)
obj = People('albert', 18, 'male')
print(obj)  # print(obj.__str__())

<名字：albert 年龄：18 性别：male>


2. call方法
call方法是在调用对象的时候自动执行（等下会用）

In [7]:
class Foo:
    def __init__(self):
        pass
    def __str__(self):
        return '123123'
    def __del__(self):
        pass
    # 调用对象，则会自动触发对象下的绑定方法__call__的执行，
    # 然后将对象本身当作第一个参数传给self，将调用对象时括号内的值
    # 传给*args与**kwargs
    def __call__(self, *args, **kwargs):
        print('__call__', args, kwargs)
obj = Foo()
# print(obj)
obj(1, 2, 3, a=1, b=2, c=3)

__call__ (1, 2, 3) {'a': 1, 'b': 2, 'c': 3}


## 三 元类
1. 补充exec的用法
exec()可以用来执行字符串内的代码，需要传三个参数，格式exec(字符串代码，全局名称空间，局部名称空间)

In [11]:
# 注意：字符串内的代码没有声明是全局的，默认就是局部的，这和我们常规的代码不同
code="""
x=1
y=2
"""
global_dic={}
local_dic={}
exec(code,global_dic,local_dic)
# print(global_dic)  # 打印的东西很多，其实就是一个字典，这些都是内置的
print(local_dic)

{'x': 1, 'y': 2}


In [12]:
# 如果我们现在自己定义全局，然后在把它在code里面改了，如下

code="""
global x  # 先声明全局，再修改
x=111
y=2
"""
global_dic={'x':999}
local_dic={}
exec(code,global_dic,local_dic)
# print(global_dic)  # 重点看字典的第一组键值对
print(local_dic)

{'y': 2}


自定义元类 
作业要求：
自定义轿车元类CarMeta，实现元类为CarMeta的类至少有
生产日期(production_date)、
发动机编号(engine_number)
及载客量(capacity)三个基本属性，没有就不行

In [41]:

class CarMeta(type):
    def __init__(self,class_name,class_bases,class_dic):
        dic=class_dic
        if not ('production_date' in dic and 'engine_number' in dic and 'capacity' in dic):
            raise AttributeError('必须有生产日期、发动机编号及载客量')
            
class Car(metaclass=CarMeta):
    production_date="production_date"
    engine_number="engine_number"
    

AttributeError: 必须有生产日期、发动机编号及载客量

# 单例模式
单例模式：顾名思义就是单个实例的意思，我们使用单例模式就是为了节省内存空间。

今天我们就来讲单例模式，那么什么场景下会用到单例模式来节省内存空间呢？

In [42]:
class MySQL:
    def __init__(self,ip,port):
        self.ip=ip
        self.port=port
obj1=MySQL('1.1.1.2',3306)
obj2=MySQL('1.1.1.3',3307)
obj3=MySQL('1.1.1.4',3308)
print(obj1)
print(obj2)
print(obj3)

三个obj明显不是一个内存地址，因为对实例化了三次就会需要三个对象的名称空间来存他们各自的属性，但是我们存不是目的，目的是为了取，取得时候确实一样的，这样显然浪费了内存空间而毫无意义

In [47]:
# 使用模块
class Singleton(object):
    def foo(self):
        pass
singleton = Singleton()

In [52]:
# 使用装饰器
def Singleton(cls):
    _instance = {}

    def _singleton(*args, **kargs):
        if cls not in _instance:
            _instance[cls] = cls(*args, **kargs)
        return _instance[cls]

    return _singleton


@Singleton
class A(object):
    a = 1

    def __init__(self, x=0):
        self.x = x


a1 = A(2)
a2 = A(3)
a1,a2

(<__main__.A at 0x2a120b718d0>, <__main__.A at 0x2a120b718d0>)

In [50]:
# 使用类
class Singleton(object):

    def __init__(self):
        pass

    @classmethod
    def instance(cls, *args, **kwargs):
        if not hasattr(Singleton, "_instance"):
            Singleton._instance = Singleton(*args, **kwargs)
        return Singleton._instance

In [51]:
# 基于new
import threading
class Singleton(object):
    _instance_lock = threading.Lock()

    def __init__(self):
        pass


    def __new__(cls, *args, **kwargs):
        if not hasattr(Singleton, "_instance"):
            with Singleton._instance_lock:
                if not hasattr(Singleton, "_instance"):
                    Singleton._instance = object.__new__(cls)  
        return Singleton._instance

obj1 = Singleton()
obj2 = Singleton()
print(obj1,obj2)

def task(arg):
    obj = Singleton()
    print(obj)

for i in range(10):
    t = threading.Thread(target=task,args=[i,])
    t.start()


<__main__.Singleton object at 0x000002A11F0ED0B8> <__main__.Singleton object at 0x000002A11F0ED0B8>
<__main__.Singleton object at 0x000002A11F0ED0B8><__main__.Singleton object at 0x000002A11F0ED0B8>

